In [1]:
from llm_foundation import logger
from llm_foundation.basic_structs import Provider, LMConfig
from llm_foundation.extractors import PlanExtractor, Plan, MultiTreePlan, Step
from llm_foundation.lm import get_model_catalog, get_lm
from llm_foundation.routing import ToolMaster
from llm_foundation.utils import banner, show_banner
from langchain.agents.output_parsers.tools import ToolsAgentOutputParser
from typing import Union, List, Dict, Any
from langchain.prompts import ChatPromptTemplate

2024-08-28 01:32:00 fperez-gcloud-stupid-sailor-twift root[1541729] INFO Logger root configured


Trying to configure logger root in module llm_foundation
root # of associated handlers - 0
Logging is not configured yet. Configuring it now.
Basic logging config


2024-08-28 01:32:01 fperez-gcloud-stupid-sailor-twift langchain_community.utils.user_agent[1541729] WARNING USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
lm_config = LMConfig(model="gpt-4o-mini", provider=Provider.LC)
lm = get_lm(lm_config)

2024-08-28 01:32:01 fperez-gcloud-stupid-sailor-twift root[1541729] INFO Creating lm object for provider LC
2024-08-28 01:32:01 fperez-gcloud-stupid-sailor-twift root[1541729] INFO Creating ModelType.Chat object for model gpt-4o-mini
2024-08-28 01:32:01 fperez-gcloud-stupid-sailor-twift root[1541729] INFO LM object client=<openai.resources.chat.completions.Completions object at 0x7f6dc5b5c650> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f6dc5b5db20> root_client=<openai.OpenAI object at 0x7f6dc5e485c0> root_async_client=<openai.AsyncOpenAI object at 0x7f6dc5b5c680> model_name='gpt-4o-mini' temperature=0.0 openai_api_key=SecretStr('**********') openai_proxy='' max_tokens=300 created


In [3]:
plan = """"
Thought: To determine the combined weight of a Toy Poodle and a Scottish Terrier, I need to find the average weight of each breed and then sum them up.

Plan:
1. Find the average weight of a Toy Poodle.
2. Find the average weight of a Scottish Terrier.
3. Sum the average weights of both breeds to get the combined weight.

Let's proceed with the plan.
"""

plan_extractor_tool = PlanExtractor.build_tool(lm, Plan, use_pydantic_output=False)

print(type(plan_extractor_tool))
print(plan_extractor_tool.name)
print(plan_extractor_tool.description)
print(plan_extractor_tool.args)

<class 'langchain_core.tools.structured.StructuredTool'>
extract
Extracts from the text passed an itemized list of plan steps as text.
{'text': {'title': 'Text', 'type': 'string'}}


In [4]:
tools = [plan_extractor_tool]

In [5]:

query = "calculate 12 * 4"
query_requiring_tools = f"extract the plan from {plan}"

print("\nQuery without tools")
print(lm.invoke(query))

lm_with_tools = lm.bind_tools(tools)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])

print("\nQuery with tools")
chain = prompt | lm_with_tools # | ToolsAgentOutputParser()
chain_response = chain.invoke({"input": query_requiring_tools})
print(type(chain_response))
print(chain_response)

print("\nQuery with tools with tools output parser")
chain = prompt | lm_with_tools |ToolsAgentOutputParser()
chain.invoke({"input": query_requiring_tools})


Query without tools


2024-08-28 01:32:02 fperez-gcloud-stupid-sailor-twift httpx[1541729] INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


content='12 * 4 = 48.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 13, 'total_tokens': 21}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None} id='run-cddcb1af-0c92-4bcc-8973-b32b2c025bcb-0' usage_metadata={'input_tokens': 13, 'output_tokens': 8, 'total_tokens': 21}

Query with tools


2024-08-28 01:32:04 fperez-gcloud-stupid-sailor-twift httpx[1541729] INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


<class 'langchain_core.messages.ai.AIMessage'>
content='' additional_kwargs={'tool_calls': [{'id': 'call_LVoVWuM0ANsSSxRJlWAGf0V1', 'function': {'arguments': '{"text":"Thought: To determine the combined weight of a Toy Poodle and a Scottish Terrier, I need to find the average weight of each breed and then sum them up.\\n\\nPlan:\\n1. Find the average weight of a Toy Poodle.\\n2. Find the average weight of a Scottish Terrier.\\n3. Sum the average weights of both breeds to get the combined weight.\\n\\nLet\'s proceed with the plan."}', 'name': 'extract'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 98, 'prompt_tokens': 144, 'total_tokens': 242}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_9722793223', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-c7658567-9f27-49e1-b353-ad324ea59565-0' tool_calls=[{'name': 'extract', 'args': {'text': "Thought: To determine the combined weight of a Toy Poodle and a Scottis

2024-08-28 01:32:06 fperez-gcloud-stupid-sailor-twift httpx[1541729] INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[ToolAgentAction(tool='extract', tool_input={'text': "Thought: To determine the combined weight of a Toy Poodle and a Scottish Terrier, I need to find the average weight of each breed and then sum them up.\n\nPlan:\n1. Find the average weight of a Toy Poodle.\n2. Find the average weight of a Scottish Terrier.\n3. Sum the average weights of both breeds to get the combined weight.\n\nLet's proceed with the plan."}, log='\nInvoking: `extract` with `{\'text\': "Thought: To determine the combined weight of a Toy Poodle and a Scottish Terrier, I need to find the average weight of each breed and then sum them up.\\n\\nPlan:\\n1. Find the average weight of a Toy Poodle.\\n2. Find the average weight of a Scottish Terrier.\\n3. Sum the average weights of both breeds to get the combined weight.\\n\\nLet\'s proceed with the plan."}`\n\n\n', message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_jIhEAw5ebd12RQXRNwpLnaKE', 'function': {'arguments': '{"text":"Thought: To de

# Tool Master

In [6]:
tool_master = ToolMaster(tools)

In [7]:
new_chain = prompt | lm_with_tools | tool_master
chain_response = new_chain.invoke({"input": query_requiring_tools})
chain_response


2024-08-28 01:32:08 fperez-gcloud-stupid-sailor-twift httpx[1541729] INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-08-28 01:32:08 fperez-gcloud-stupid-sailor-twift root[1541729] INFO ========================================================================================================================
2024-08-28 01:32:08 fperez-gcloud-stupid-sailor-twift root[1541729] INFO =                                                      Call Tool                                                       =
2024-08-28 01:32:08 fperez-gcloud-stupid-sailor-twift root[1541729] INFO ========================================================================================================================
2024-08-28 01:32:08 fperez-gcloud-stupid-sailor-twift root[1541729] INFO Extracting <class 'llm_foundation.extractors.Plan'>


Type plan: <class 'llm_foundation.extractors.Plan'>


2024-08-28 01:32:09 fperez-gcloud-stupid-sailor-twift httpx[1541729] INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-08-28 01:32:09 fperez-gcloud-stupid-sailor-twift root[1541729] INFO Response (<class 'dict'>): {'plan': [{'id': '1', 'description': 'Find the average weight of a Toy Poodle.', 'dependend_steps': ['3'], 'depending_steps': []}, {'id': '2', 'description': 'Find the average weight of a Scottish Terrier.', 'dependend_steps': ['3'], 'depending_steps': []}, {'id': '3', 'description': 'Sum the average weights of both breeds to get the combined weight.', 'dependend_steps': [], 'depending_steps': ['1', '2']}]}


[('call_HeZotnjhcRiuzYMSXH05KU2I',
  'extract',
  {'plan': [{'id': '1',
     'description': 'Find the average weight of a Toy Poodle.',
     'dependend_steps': ['3'],
     'depending_steps': []},
    {'id': '2',
     'description': 'Find the average weight of a Scottish Terrier.',
     'dependend_steps': ['3'],
     'depending_steps': []},
    {'id': '3',
     'description': 'Sum the average weights of both breeds to get the combined weight.',
     'dependend_steps': [],
     'depending_steps': ['1', '2']}]})]

In [8]:
print(chain_response[0][2])
print(type(chain_response[0][2]))

{'plan': [{'id': '1', 'description': 'Find the average weight of a Toy Poodle.', 'dependend_steps': ['3'], 'depending_steps': []}, {'id': '2', 'description': 'Find the average weight of a Scottish Terrier.', 'dependend_steps': ['3'], 'depending_steps': []}, {'id': '3', 'description': 'Sum the average weights of both breeds to get the combined weight.', 'dependend_steps': [], 'depending_steps': ['1', '2']}]}
<class 'dict'>


In [9]:
plan_data = chain_response[0][2]
print(plan_data)

{'plan': [{'id': '1', 'description': 'Find the average weight of a Toy Poodle.', 'dependend_steps': ['3'], 'depending_steps': []}, {'id': '2', 'description': 'Find the average weight of a Scottish Terrier.', 'dependend_steps': ['3'], 'depending_steps': []}, {'id': '3', 'description': 'Sum the average weights of both breeds to get the combined weight.', 'dependend_steps': [], 'depending_steps': ['1', '2']}]}


In [10]:
multi_tree = PlanExtractor.build_multi_tree_plan(plan_data)

In [11]:
multi_tree.traverse_leafs_first_dependants()

[Step(id='1', description='Find the average weight of a Toy Poodle.', dependend_steps=['3'], depending_steps=[]),
 Step(id='2', description='Find the average weight of a Scottish Terrier.', dependend_steps=['3'], depending_steps=[]),
 Step(id='3', description='Sum the average weights of both breeds to get the combined weight.', dependend_steps=[], depending_steps=[Step(id='1', description='Find the average weight of a Toy Poodle.', dependend_steps=['3'], depending_steps=[]), Step(id='2', description='Find the average weight of a Scottish Terrier.', dependend_steps=['3'], depending_steps=[])])]

In [12]:
multi_tree.traverse_depth_first()

3
1
2


In [13]:
multi_tree.traverse_breadth_first()


3
1
2


In [15]:
import json
pydantic_json_plan = '''
{
    "plan": [
        {"id": "1", "description": "Task 1", "dependent_steps": [], "depending_steps": ["2", "3"]},
        {"id": "2", "description": "Task 2", "dependent_steps": ["1", "5"], "depending_steps": ["4"]},
        {"id": "3", "description": "Task 3", "dependent_steps": ["1"], "depending_steps": []},
        {"id": "4", "description": "Task 4", "dependent_steps": ["2"], "depending_steps": []},
        {"id": "5", "description": "Task 5", "dependent_steps": [], "depending_steps": ["2"]}
    ]
}
'''
json_plan=json.loads(pydantic_json_plan)
fake_multi_tree = PlanExtractor.build_multi_tree_plan(json_plan)

fake_multi_tree.traverse_leafs_first_dependants()

ValidationError: 5 validation errors for Plan
plan -> 0 -> dependend_steps
  field required (type=value_error.missing)
plan -> 1 -> dependend_steps
  field required (type=value_error.missing)
plan -> 2 -> dependend_steps
  field required (type=value_error.missing)
plan -> 3 -> dependend_steps
  field required (type=value_error.missing)
plan -> 4 -> dependend_steps
  field required (type=value_error.missing)

In [ ]:

fake_multi_tree.traverse_depth_first()


In [ ]:
fake_multi_tree.traverse_breadth_first()